# TCGA-UCEC – Notebook 01
## Chargement des données et Sanity check

Objectif : charger les données RNA-seq et cliniques TCGA-UCEC depuis `data/raw/` et effectuer les premiers contrôles intégrité (dimensions, cohérence des identifiants, valeurs manquantes).

## Fiche d’identité des jeux de données bruts (TCGA-UCEC)

Les données utilisées dans ce projet proviennent exclusivement du **Genomic Data Commons (GDC)** via la plateforme **UCSC Xena**, conformément aux consignes du projet.

---

### 1. Données cliniques (Phenotype)

- **Dataset** : Phenotype  
- **Cohorte** : TCGA Endometrioid Cancer (UCEC)  
- **Dataset ID** : `TCGA-UCEC.clinical.tsv`  
- **Hub Xena** : https://gdc.xenahubs.net  
- **Source officielle** : Genomic Data Commons (GDC)  
- **Auteur** : Genomic Data Commons  
- **Version** : 09-07-2024  
- **Type de données** : Phénotype / clinique  

**Accès aux données :**
- Téléchargement : https://gdc-hub.s3.us-east-1.amazonaws.com/download/TCGA-UCEC.clinical.tsv.gz  
- Notes de release GDC : https://docs.gdc.cancer.gov/Data/Release_Notes/Data_Release_Notes/#data-release-410  
- API GDC : https://api.gdc.cancer.gov/data/

**Format des données :**
- Matrice **lignes = échantillons / patients**
- Colonnes = variables cliniques
- **Dimensions** : 600 échantillons × 79 variables cliniques

Ces données contiennent notamment les informations nécessaires à la
caractérisation clinique et aux sous-types moléculaires TCGA.

---

### 2. Données transcriptomiques (RNA-seq – STAR counts)

- **Dataset** : Gene expression RNA-seq – STAR counts  
- **Cohorte** : TCGA Endometrioid Cancer (UCEC)  
- **Dataset ID** : `TCGA-UCEC.star_counts.tsv`  
- **Hub Xena** : https://gdc.xenahubs.net  
- **Source officielle** : Genomic Data Commons (GDC)  
- **Auteur** : Genomic Data Commons  
- **Version** : 05-10-2024  
- **Type de données** : Expression génique RNA-seq  
- **Plateforme** : Illumina  
- **Unité** : log2(count + 1)

**Accès aux données :**
- Téléchargement : https://gdc-hub.s3.us-east-1.amazonaws.com/download/TCGA-UCEC.star_counts.tsv.gz  
- Pipeline RNA-seq GDC : https://docs.gdc.cancer.gov/Data/Bioinformatics_Pipelines/Expression_mRNA_Pipeline/  
- Notes de release GDC : https://docs.gdc.cancer.gov/Data/Release_Notes/Data_Release_Notes/#data-release-400  
- API GDC : https://api.gdc.cancer.gov/data/

**Annotation génomique :**
- GENCODE v36  
- Mapping gènes : https://gdc-hub.s3.us-east-1.amazonaws.com/download/gencode.v36.annotation.gtf.gene.probemap

**Traitement (wrangling) effectué par Xena :**
- Les données provenant de différents vials / portions / aliquots
  pour un même échantillon sont **moyennées**
- Les valeurs sont ensuite transformées en **log2(x + 1)**

**Format des données :**
- Matrice **lignes = gènes**
- Colonnes = échantillons
- **Dimensions** : 60 661 gènes × 585 échantillons

---

### Remarque importante

Les deux jeux de données sont fournis à des niveaux différents
(expression au niveau échantillon, clinique au niveau patient),
ce qui justifie l’étape d’harmonisation des identifiants TCGA
réalisée dans les notebooks suivants.


## 1. Configuration et imports

In [1]:
# ==========================================================================================================
import os                                       # Navigation fichiers (DIRS, chemins relatifs)
import warnings                                 # Masquer warnings (dépréciation)
import gc                                       # Gestion mémoire (nettoyage objets inutilisés)
import json                                     # Lecture du dictionnaire de métadonnées

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

# ==========================================================================================================
PROJECT_ROOT = r"C:\Z\M2_AIDA\TCGA_UCEC_project" #Laïla

DIRS = {
    # Racine data
    "DATA": os.path.join(PROJECT_ROOT, "data"),

    # Données
    "RAW": os.path.join(PROJECT_ROOT, "data", "raw"),

    # Données intermédiaires (persistées)
    "PROCESSED":        os.path.join(PROJECT_ROOT, "data", "processed"),
    "NORM":             os.path.join(PROJECT_ROOT, "data", "processed", "normalized"),
    "QC_FILTERED":      os.path.join(PROJECT_ROOT, "data", "processed", "qc_filtered"),
    "COHORT_FILTERED":  os.path.join(PROJECT_ROOT, "data", "processed", "cohort_filtered"),
    "SC_OBJECTS":       os.path.join(PROJECT_ROOT, "data", "processed", "single_cell_objects"),
    "PSEUDOBULK_PROC":  os.path.join(PROJECT_ROOT, "data", "processed", "pseudobulk"),

    # Artefacts (par étape du pipeline)
    "ARTEFACTS": os.path.join(PROJECT_ROOT, "data", "artefacts"),

    "EDA":         os.path.join(PROJECT_ROOT, "data", "artefacts", "exploratory_data_analysis"),
    "QC":          os.path.join(PROJECT_ROOT, "data", "artefacts", "qc_analysis"),
    "COHORT":      os.path.join(PROJECT_ROOT, "data", "artefacts", "cohort_selection"),
    "SINGLE_CELL": os.path.join(PROJECT_ROOT, "data", "artefacts", "single_cell_analysis"),
    "PSEUDOBULK":  os.path.join(PROJECT_ROOT, "data", "artefacts", "pseudobulk_preparation"),
    "DE":          os.path.join(PROJECT_ROOT, "data", "artefacts", "differential_expression"),
    "ENRICH":      os.path.join(PROJECT_ROOT, "data", "artefacts", "functional_enrichment"),

    # Autres dossiers du projet
    "RESULTS_R": os.path.join(PROJECT_ROOT, "Results_R_Analysis"),
    "TMP":       os.path.join(PROJECT_ROOT, "tmp_cache"),
    "DOCS":      os.path.join(PROJECT_ROOT, "documentation"),
}
for path in DIRS.values():
    os.makedirs(path, exist_ok=True)
os.chdir(PROJECT_ROOT)

# ==========================================================================================================
warnings.filterwarnings("ignore") 
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (10, 6)

# ==========================================================================================================
print(f"✅ Environnement chargé. Working directory: {os.getcwd()}")
print(f"✅ Dossiers créés:")
for name, path in DIRS.items():
    print(f"   {name}: {path}")


✅ Environnement chargé. Working directory: c:\Z\M2_AIDA\TCGA_UCEC_project
✅ Dossiers créés:
   DATA: C:\Z\M2_AIDA\TCGA_UCEC_project\data
   RAW: C:\Z\M2_AIDA\TCGA_UCEC_project\data\raw
   PROCESSED: C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed
   NORM: C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\normalized
   QC_FILTERED: C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\qc_filtered
   COHORT_FILTERED: C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\cohort_filtered
   SC_OBJECTS: C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\single_cell_objects
   PSEUDOBULK_PROC: C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\pseudobulk
   ARTEFACTS: C:\Z\M2_AIDA\TCGA_UCEC_project\data\artefacts
   EDA: C:\Z\M2_AIDA\TCGA_UCEC_project\data\artefacts\exploratory_data_analysis
   QC: C:\Z\M2_AIDA\TCGA_UCEC_project\data\artefacts\qc_analysis
   COHORT: C:\Z\M2_AIDA\TCGA_UCEC_project\data\artefacts\cohort_selection
   SINGLE_CELL: C:\Z\M2_AIDA\TCGA_UCEC_project\data\artefacts\single_cell_analysis
   PSEUDOBULK

## 2. Chargement des données brutes

In [2]:
expr_filename = "TCGA-UCEC.star_counts.tsv.gz"
clin_filename = "TCGA-UCEC.clinical.tsv.gz"

expr_path = os.path.join(DIRS["RAW"], expr_filename)
clin_path = os.path.join(DIRS["RAW"], clin_filename)

expr = pd.read_csv(expr_path, sep='\t')
clin = pd.read_csv(clin_path, sep='\t')

# ==========================================================================================================
print(
    f"✅ Datasets successfully loaded\n"
    f"   - Expression matrix: {expr_path} | shape (genes × samples): {expr.shape}\n"
    f"   - Clinical matrix:   {clin_path} | shape (samples × variables): {clin.shape}"
)

✅ Datasets successfully loaded
   - Expression matrix: C:\Z\M2_AIDA\TCGA_UCEC_project\data\raw\TCGA-UCEC.star_counts.tsv.gz | shape (genes × samples): (60660, 586)
   - Clinical matrix:   C:\Z\M2_AIDA\TCGA_UCEC_project\data\raw\TCGA-UCEC.clinical.tsv.gz | shape (samples × variables): (600, 79)


## 3. Vérification des identifiants échantillons

In [3]:
expr_samples = set(expr.columns)
clin_samples = set(clin.index)

print(f'Nombre d échantillons RNA-seq : {len(expr_samples)}')
print(f'Nombre d échantillons cliniques : {len(clin_samples)}')

Nombre d échantillons RNA-seq : 586
Nombre d échantillons cliniques : 600


## 4. Aperçu des données

In [4]:
print(
    "✅ Aperçu de la matrice d'expression RNA-seq :\n"
    "   - lignes : gènes (ID Ensembl)\n"
    "   - colonnes : échantillons TCGA\n"
    "   - affichage : 5 premiers gènes × 5 premiers échantillons\n"
)

display(
    expr.iloc[:5, :5].rename_axis("gene_id")
)


✅ Aperçu de la matrice d'expression RNA-seq :
   - lignes : gènes (ID Ensembl)
   - colonnes : échantillons TCGA
   - affichage : 5 premiers gènes × 5 premiers échantillons



,Ensembl_ID,TCGA-FI-A3PX-01A,TCGA-BG-A221-01A,TCGA-EY-A1GK-01A,TCGA-BG-A2AE-01A
gene_id,,,,,
0,ENSG00000000003.15,12.671541,10.674192,10.895575,10.540128
1,ENSG00000000005.6,3.000000,0.000000,1.584963,2.321928
2,ENSG00000000419.13,11.692616,9.321928,9.247928,8.807355
3,ENSG00000000457.14,8.945444,7.936638,9.049849,8.199672
4,ENSG00000000460.17,9.337622,7.700440,7.118941,6.820179


In [4]:
print(
    "✅ Aperçu des données cliniques (phenotype) :\n"
    "   - lignes : échantillons TCGA\n"
    "   - colonnes : variables cliniques et annotations\n"
    "   - affichage : 5 premiers échantillons\n"
)

display(clin.head())


✅ Aperçu des données cliniques (phenotype) :
   - lignes : échantillons TCGA
   - colonnes : variables cliniques et annotations
   - affichage : 5 premiers échantillons



,sample,id,disease_type,case_id,submitter_id,primary_site,alcohol_history.exposures,race.demographic,gender.demographic,ethnicity.demographic,...,sample_type.samples,composition.samples,days_to_collection.samples,initial_weight.samples,preservation_method.samples,pathology_report_uuid.samples,oct_embedded.samples,specimen_type.samples,is_ffpe.samples,tissue_type.samples
0,TCGA-AJ-A3NC-01A,c8fb94dd-c535-48f4-9419-aac2bf47bbb4,Adenomas and Adenocarcinomas,c8fb94dd-c535-48f4-9419-aac2bf47bbb4,TCGA-AJ-A3NC,Corpus uteri,Not Reported,white,female,not reported,...,Primary Tumor,Not Reported,282.0,300.0,Unknown,B6DB87F4-E88A-4209-92E2-75ADB364FB28,False,Solid Tissue,False,Tumor
1,TCGA-AJ-A3NC-11A,c8fb94dd-c535-48f4-9419-aac2bf47bbb4,Adenomas and Adenocarcinomas,c8fb94dd-c535-48f4-9419-aac2bf47bbb4,TCGA-AJ-A3NC,Corpus uteri,Not Reported,white,female,not reported,...,Solid Tissue Normal,Not Reported,282.0,70.0,Unknown,NaN,False,Solid Tissue,False,Normal
2,TCGA-KP-A3W0-01A,c9ebaeff-6a27-4a6c-a34c-5337a3508a52,"Cystic, Mucinous and Serous Neoplasms",c9ebaeff-6a27-4a6c-a34c-5337a3508a52,TCGA-KP-A3W0,Corpus uteri,Not Reported,not reported,female,not reported,...,Primary Tumor,Not Reported,351.0,90.0,OCT,8623B627-C3E3-469B-A640-2F8BCFC0B711,True,Solid Tissue,False,Tumor
3,TCGA-B5-A11N-01A,ca594f56-9c68-4b7e-84b2-669f1004e3a4,Adenomas and Adenocarcinomas,ca594f56-9c68-4b7e-84b2-669f1004e3a4,TCGA-B5-A11N,Corpus uteri,Not Reported,white,female,not reported,...,Primary Tumor,Not Reported,1140.0,370.0,OCT,A4C1DE15-A45A-4D31-B363-C6652370C09C,True,Solid Tissue,False,Tumor
4,TCGA-BS-A0V7-01A,ca858743-1f06-40e3-accf-395d08375b63,Adenomas and Adenocarcinomas,ca858743-1f06-40e3-accf-395d08375b63,TCGA-BS-A0V7,Corpus uteri,Not Reported,native hawaiian or other pacific islander,female,not hispanic or latino,...,Primary Tumor,Not Reported,1455.0,130.0,Unknown,A4515806-9888-48E8-8F9E-491527FFC29C,False,Solid Tissue,False,Tumor


## 5. Prochaines étapes
- Harmonisation des identifiants échantillons
- Identification des labels de sous-types TCGA
- QC échantillons (outliers globaux)